# Imports

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model, set_random_seed

# Developing Model

## Design Sequential Model

In [ ]:
set_random_seed(999)

In [ ]:
def create_sequential_model(vocab_size, embedding_dim, sequence_length, lstm_out_dim):
    # initialize a sequential model
    model = Sequential([
                Input(shape=(sequence_length,)),
                Embedding(input_dim=vocab_size, output_dim=embedding_dim),
                Bidirectional(LSTM(units=lstm_out_dim)),
                Dense(units=10, activation='relu'),
                Dense(units=1, activation='sigmoid')
            ])
    
    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model   

In [ ]:
# create model
model_seq = create_sequential_model(vocab_size=10000, embedding_dim=16, sequence_length=200, lstm_out_dim=32)

# print model summary
print(model_seq.summary())

In [ ]:
# plot model
# plot_model(model_seq)

## Creating Model Checkpoints

In [ ]:
checkpoint_path = os.path.join(os.getcwd(), "training/cp.ckpt/imdb_sentiment_best_model.weights.h5")
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)

In [ ]:
# checkpoint_filepath = os.path.join(os.getcwd(), 'model.keras')

# Create a callback that saves the model's weights
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True, 
    monitor='val_loss', 
    mode='min', 
    save_best_only=True
)

callbacks = [EarlyStopping(patience=2), model_checkpoint_callback]

In [ ]:
# load training data
path_preprocessed = os.path.join(os.getcwd(), 'data-preprocessed')

with open(os.path.join(path_preprocessed, 'train_padded.pickle'), 'rb') as f:
    train_padded = pickle.load(f)
    
with open(os.path.join(path_preprocessed, 'y_train.pickle'), 'rb') as f:
    y_train = pickle.load(f)
    
with open(os.path.join(path_preprocessed, 'test_padded.pickle'), 'rb') as f:
    test_padded = pickle.load(f)
    
with open(os.path.join(path_preprocessed, 'y_test.pickle'), 'rb') as f:
    y_test = pickle.load(f)

In [ ]:
history = model_seq.fit(
    train_padded, 
    y_train, 
    epochs=10, 
    validation_data=(test_padded, y_test),
    callbacks=[model_checkpoint_callback]
)

In [ ]:
model_seq.save('imdb_model.keras')

## Get Metrics and Performance Graphs

In [ ]:
metrics_df = pd.DataFrame(history.history)
print(metrics_df)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(metrics_df.index, metrics_df.loss)
plt.plot(metrics_df.index, metrics_df.val_loss)
plt.title('IMBD Reviews Sentiment Analysis Model Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(metrics_df.index, metrics_df.accuracy)
plt.plot(metrics_df.index, metrics_df.val_accuracy)
plt.title('IMBD Reviews Sentiment Analysis Model Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.show()

# Sentiment Prediction Using the Trained Model

## Load Tokenizer

In [ ]:
with open(os.path.join(path_preprocessed, 'tokenizer.pickle'), 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
# tokenizer.word_index

In [ ]:
# predict on a sample text without padding.
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')

print([sample_text])

In [ ]:
# tokenize reviews
sequence_length = 200
sample_sequences = tokenizer.texts_to_sequences([sample_text])
sample_sequences_padded = np.array(
            pad_sequences(
                sample_sequences, 
                maxlen=sequence_length, 
                padding='post', 
                truncating='post'
            )
        )

x_test = sample_sequences_padded.reshape(1, sequence_length)

## Load Model

In [ ]:
# trained_model = create_sequential_model(vocab_size=10000, embedding_dim=16, sequence_length=200, lstm_out_dim=32)
# trained_model.load_weights(checkpoint_path)

In [ ]:
trained_model = load_model('/workspace/nlp-projects/lstm-text-classification/imdb_model.keras')

In [ ]:
# model by loading weights
predictions = trained_model.predict(x_test)
print(predictions.item())

In [ ]:
# model from memory
predictions = model_seq.predict(x_test)
print(predictions.item())

# References

- https://www.embedded-robotics.com/sentiment-analysis-using-lstm/
- https://www.tensorflow.org/tutorials/keras/save_and_load